## Import Library

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv


## Read Input Files

In [2]:
train_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa', 'train_features.csv'))
test_data = pd.read_csv(os.path.join('/kaggle/input/lish-moa' , 'test_features.csv'))

train_target = pd.read_csv(os.path.join('/kaggle/input/lish-moa','train_targets_scored.csv'))
sample = pd.read_csv(os.path.join('/kaggle/input/lish-moa' , 'sample_submission.csv'))

## Split data to train and test

In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data , train_target, test_target  = train_test_split(train_data,train_target, test_size=0.2)

## Print Shape of Data

In [4]:
print("train_data.shap: {}  , train_target.shap: {}  , test_data.shap: {} ".format(train_data.shape , train_target.shape , test_data.shape))

train_data.shap: (19051, 876)  , train_target.shap: (19051, 207)  , test_data.shap: (4763, 876) 


In [5]:
NGTypeFeature = sum(train_data.columns.to_series().str.contains('g-') == True )
NCTypeFeature = sum(train_data.columns.to_series().str.contains('c-') == True )

print("NGTypeFeature = {} \nNCTypeFeature = {}".format(NGTypeFeature , NCTypeFeature))

NGTypeFeature = 772 
NCTypeFeature = 100


## Define a Neural Network To Train

In [6]:
import torch.nn.functional as F
import torch.nn as nn

class MultiLabelClassifer(nn.Module):
    #TO-DO: Documentation
    def __init__(self , input_feature , hidden_dim , output_dim):
        super(MultiLabelClassifer , self).__init__()
        
        #nn.utils.weight_norm
        self.fc1 = (nn.Linear(in_features = input_feature , out_features = hidden_dim))
        #TO-DO: As close features are related here, convoluation layer will help here
        #TO-DO: Later add and play with Convolutatin layer, to see accurracy difference
        self.fc2 =  (nn.Linear( in_features = hidden_dim , out_features  = hidden_dim))
        self.fc3 = (nn.Linear( in_features = hidden_dim , out_features  = hidden_dim))
        self.fc4 =  (nn.Linear(in_features = hidden_dim , out_features = hidden_dim))
        self.fc5 = (nn.Linear(in_features = hidden_dim , out_features = output_dim))
        self.drop = nn.Dropout(0.4) # dropout with 30% prob
        
        self.out = (nn.Linear(output_dim,output_dim)) #Above using weight normalization for final layer
        #Query: Why use weight normalization. Checkout this awesome stackoverlfow post:
        #link: https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
        
        self.sig = nn.Sigmoid()
        #self.F_Sig = F.sigmoid() #signout output range from [0,1]
        self.Log_Sig = nn.LogSigmoid() #log of range 0-1 is whole number?
        self.soft = nn.Softmax()  #sum of all output is '1'. So its used only in multi-class problem.
        #self.linear3 = nn.utils.weight_norm(nn.Linear(hidden_size,output_size))
        #TO-DO: Define output layer so that it be as per Question requirement
    def forward(self, x):
        
        # (1) input layer
        
        t = x
        
        
        # (2) Hidden Linear Layer
        
        t = self.fc1(t)
        t = F.relu(t)
        #3
        
        t = self.fc2(t)
        t = F.relu(t)
        #4
        
        t= self.fc3(t)
        t = F.relu(t)
        t = self.drop(t)
        
        #5
        
        t = self.fc4(t)
        t = F.relu(t)
        #6
        
        t = self.drop(t)
        
        t = self.fc5(t)
        #t = F.relu(t)
        #t = self.F_Sig(t)
        #t = F.sigmoid(t)
        t = self.Log_Sig(t) #Works, if no activation is used after fc5
        
        # (7) Output Layer
        
        #t = self.out(t) #This working okay. but why? 
        
        #t = self.sig(t)  #Using Sigmmoid ALSO as last layer is causing training loss to not decrease
        #t=self.soft(t) #==> Using softmax as last layer is causing training loss to not decrease
        
        #t = self.Log_Sig(t)
        
        return t
        

## Make Copy of data to Work ON

In [7]:
train_nn = train_data.copy()
train_data.head()

test_nn = test_data.copy()
train_nn.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
12696,id_88cbf9e6f,trt_cp,24,D1,-0.0363,-0.7052,2.4370,0.4958,-0.0288,0.9325,...,0.8354,0.6232,1.3220,0.7226,1.5900,0.9043,0.5915,0.6916,0.4090,1.3070
15178,id_a31fe4c72,trt_cp,24,D2,-0.5555,-0.2524,-0.0457,-1.0740,-0.2493,-0.1253,...,0.6307,-0.1247,1.0530,0.8649,-1.1550,-0.0001,-0.3790,-0.3946,0.6762,-0.2142
23144,id_f89cb0273,trt_cp,48,D2,4.1270,0.3536,3.1270,2.5220,1.0280,-1.9550,...,-10.0000,-10.0000,-8.8670,-10.0000,-10.0000,-8.8740,-10.0000,-7.2720,-8.8470,-7.6540
6605,id_46a556983,trt_cp,48,D1,-0.7084,-1.6320,-0.3802,0.6172,-1.8590,0.5889,...,1.1510,-0.8152,0.7311,0.7782,0.7926,0.3287,0.8066,0.6009,-0.0479,0.1848
7014,id_4b33dbf64,ctl_vehicle,24,D2,0.2139,-1.2400,0.3305,-0.4977,-1.5110,0.0809,...,-0.1928,-1.2300,-0.0741,0.4578,0.8259,0.9920,-0.4301,0.3867,-1.0180,0.3901


## One hot coding of input feature and Normalization

In [8]:
import torch.utils.data
import torch
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,Normalizer

ignore_columns = ['sig_id','cp_type']

train_columns = [x for x in train_data.columns if x not in ignore_columns]
train = train_nn[train_columns]  #Extract Column Except ignore_columns
test = test_nn[train_columns]

#Store Original Index and Column
idx = train.index
col = train.columns

train.head()





,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
12696,24,D1,-0.0363,-0.7052,2.4370,0.4958,-0.0288,0.9325,2.0920,1.2900,...,0.8354,0.6232,1.3220,0.7226,1.5900,0.9043,0.5915,0.6916,0.4090,1.3070
15178,24,D2,-0.5555,-0.2524,-0.0457,-1.0740,-0.2493,-0.1253,1.7780,0.0968,...,0.6307,-0.1247,1.0530,0.8649,-1.1550,-0.0001,-0.3790,-0.3946,0.6762,-0.2142
23144,48,D2,4.1270,0.3536,3.1270,2.5220,1.0280,-1.9550,0.0772,-2.0710,...,-10.0000,-10.0000,-8.8670,-10.0000,-10.0000,-8.8740,-10.0000,-7.2720,-8.8470,-7.6540
6605,48,D1,-0.7084,-1.6320,-0.3802,0.6172,-1.8590,0.5889,0.1296,-1.5620,...,1.1510,-0.8152,0.7311,0.7782,0.7926,0.3287,0.8066,0.6009,-0.0479,0.1848
7014,24,D2,0.2139,-1.2400,0.3305,-0.4977,-1.5110,0.0809,-1.3840,-1.8560,...,-0.1928,-1.2300,-0.0741,0.4578,0.8259,0.9920,-0.4301,0.3867,-1.0180,0.3901


In [9]:

target = train_target.iloc[:,1:].values
#test_target = test_target[:,1:]
#https://stackoverflow.com/questions/54160370/how-to-use-sklearn-column-transformer/54160620
transform = ColumnTransformer([('o',OneHotEncoder(),[0,1]),('s',Normalizer(),list(range(3,train.shape[1])))])
print("size before transform train.shape:{} test.shape{}".format(train.shape,test.shape))
train = transform.fit_transform(train)
test = transform.transform(test)

print("size AFTER transform train.shape:{} test.shape{}".format(train.shape,test.shape))

size before transform train.shape:(19051, 874) test.shape(4763, 874)
size AFTER transform train.shape:(19051, 876) test.shape(4763, 876)


## Check Data After Transformation 

In [10]:
train_df = pd.DataFrame(data = train,index = idx ) #Check if encoding is successful or not
train_df.head()


,0,1,2,3,4,5,6,7,8,9,...,866,867,868,869,870,871,872,873,874,875
12696,1.0,0.0,0.0,1.0,0.0,-0.022241,0.076860,0.015637,-0.000908,0.029410,...,0.026348,0.019655,0.041694,0.022790,0.050147,0.028521,0.018655,0.021812,0.012899,0.041221
15178,1.0,0.0,0.0,0.0,1.0,-0.010850,-0.001965,-0.046170,-0.010717,-0.005386,...,0.027113,-0.005361,0.045267,0.037181,-0.049652,-0.000004,-0.016293,-0.016963,0.029069,-0.009208
23144,0.0,1.0,0.0,0.0,1.0,0.002474,0.021879,0.017646,0.007193,-0.013679,...,-0.069969,-0.069969,-0.062042,-0.069969,-0.069969,-0.062090,-0.069969,-0.050881,-0.061902,-0.053554
6605,0.0,1.0,0.0,1.0,0.0,-0.057470,-0.013389,0.021734,-0.065464,0.020738,...,0.040532,-0.028707,0.025745,0.027404,0.027911,0.011575,0.028404,0.021160,-0.001687,0.006508
7014,1.0,0.0,0.0,0.0,1.0,-0.042939,0.011445,-0.017235,-0.052324,0.002801,...,-0.006676,-0.042593,-0.002566,0.015853,0.028600,0.034352,-0.014894,0.013391,-0.035252,0.013509


### Observe that features are now one hot coded , and rest of the value are normalized

## Get Data Tensor

In [11]:
print(type(train))
train_data_tensor =  torch.from_numpy(train.astype(np.float32))
train_target_tensor = torch.from_numpy(target.astype(np.float32))

<class 'numpy.ndarray'>


In [12]:
print("max_val {}, min_val {}",train_data_tensor.max() , train_data_tensor.min() )
#Value range b/w -1 to 1 implies that it has been normalized

max_val {}, min_val {} tensor(1.) tensor(-0.4424)


In [13]:
print(train_target_tensor[0]) #Checking expected label for first training data

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [14]:
print(sum(sum(train_target_tensor))) #Here, it should be whole number. As output lables are not float

tensor(13440.)


### Combine features and expected label in one set

In [15]:
print("type of train_data_tensor: {}",type(train_data_tensor) )
train_set = torch.utils.data.TensorDataset(train_data_tensor , train_target_tensor)
print(train_set)

type of train_data_tensor: {} <class 'torch.Tensor'>


## Get dataloader

In [16]:
batch_size = 120
train_loader = torch.utils.data.DataLoader(train_set , batch_size = batch_size, shuffle=True)
print(train_loader)

## Now take Instance of Network

In [17]:
input_features = 876
output_featuers = 206
hidden_dimension = 4000

# def __init__(self , input_feature , hidden_dim , output_dim):
model = MultiLabelClassifer(input_features , hidden_dimension  ,output_featuers )

# Analyser Sample Formed Data

In [18]:
batch = next(iter(train_loader))
feature, label = batch

In [19]:
print(batch[0].shape)
print(batch[0][0].shape) #THis is one extracted Feature out of 100

torch.Size([120, 876])
torch.Size([876])


In [20]:
print(feature[0]) #input features for first input data point

tensor([ 1.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
        -1.5511e-02, -1.9007e-02,  2.8665e-02,  2.3071e-02,  3.6160e-03,
         6.6140e-03,  6.6232e-03, -2.7438e-02,  1.8108e-02,  8.5880e-03,
        -3.9024e-02,  2.4044e-02, -1.4252e-02, -3.3388e-02,  2.5699e-02,
         4.3775e-02,  6.0928e-03,  5.9729e-02,  2.6779e-02, -1.5101e-02,
         2.0317e-02,  1.5802e-02,  0.0000e+00,  4.2811e-02,  4.3406e-02,
        -2.5054e-02, -2.7531e-02, -3.9804e-03, -9.4090e-04,  9.0815e-02,
        -3.8466e-03,  4.9397e-02, -5.7653e-04, -3.5690e-02,  8.6664e-03,
        -2.8785e-02,  9.9533e-03, -2.5570e-02, -4.0002e-02,  5.2949e-02,
         4.9443e-02,  2.4514e-02, -2.0423e-02, -6.4110e-03,  2.2605e-02,
        -5.7653e-02,  2.6322e-02,  1.1992e-03, -2.2379e-02,  1.1162e-02,
        -7.7532e-03, -5.9175e-03,  1.2914e-03,  3.5487e-02,  4.4854e-02,
         1.3080e-02,  2.5188e-02,  1.2070e-02,  1.1544e-02,  4.0090e-02,
         1.0576e-02, -2.0423e-02,  1.9856e-02,  3.7

In [21]:
print(sum(label[0]))
print(label[0])   #label for 1st feature data

tensor(1.)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


# Verify if the weight are being updated.
## Unit test for weight updating or not.

In [22]:
print(model) #Get Model Parameters

MultiLabelClassifer(
  (fc1): Linear(in_features=876, out_features=4000, bias=True)
  (fc2): Linear(in_features=4000, out_features=4000, bias=True)
  (fc3): Linear(in_features=4000, out_features=4000, bias=True)
  (fc4): Linear(in_features=4000, out_features=4000, bias=True)
  (fc5): Linear(in_features=4000, out_features=206, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
  (out): Linear(in_features=206, out_features=206, bias=True)
  (sig): Sigmoid()
  (Log_Sig): LogSigmoid()
  (soft): Softmax(dim=None)
)


In [23]:
# Print model parameters
for param in model.parameters():
    print(param.shape)

torch.Size([4000, 876])
torch.Size([4000])
torch.Size([4000, 4000])
torch.Size([4000])
torch.Size([4000, 4000])
torch.Size([4000])
torch.Size([4000, 4000])
torch.Size([4000])
torch.Size([206, 4000])
torch.Size([206])
torch.Size([206, 206])
torch.Size([206])


In [24]:
#Print Shape
for name, param in model.named_parameters():
    print(name, '\t\t', param.shape)


fc1.weight 		 torch.Size([4000, 876])
fc1.bias 		 torch.Size([4000])
fc2.weight 		 torch.Size([4000, 4000])
fc2.bias 		 torch.Size([4000])
fc3.weight 		 torch.Size([4000, 4000])
fc3.bias 		 torch.Size([4000])
fc4.weight 		 torch.Size([4000, 4000])
fc4.bias 		 torch.Size([4000])
fc5.weight 		 torch.Size([206, 4000])
fc5.bias 		 torch.Size([206])
out.weight 		 torch.Size([206, 206])
out.bias 		 torch.Size([206])


In [25]:
print(model.fc1)
#Weight Before Model train
print("Weight Of Model Before train is: ")
print(model.fc1.weight)

Linear(in_features=876, out_features=4000, bias=True)
Weight Of Model Before train is: 
Parameter containing:
tensor([[-0.0324,  0.0287, -0.0165,  ..., -0.0183,  0.0267, -0.0308],
        [-0.0147, -0.0139,  0.0003,  ..., -0.0187, -0.0154,  0.0194],
        [-0.0293, -0.0105, -0.0217,  ...,  0.0160,  0.0012,  0.0163],
        ...,
        [ 0.0239, -0.0180,  0.0333,  ..., -0.0230, -0.0163, -0.0230],
        [-0.0122, -0.0337,  0.0221,  ..., -0.0277, -0.0140, -0.0122],
        [-0.0004,  0.0154,  0.0274,  ...,  0.0007,  0.0043, -0.0187]],
       requires_grad=True)


## Train model for one batch

In [26]:
import torch.optim as optim
batch = next(iter(train_loader))
feature, label = batch
loss_fn = nn.BCEWithLogitsLoss()

criterion = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters() , .01)

In [27]:
preds = model(feature)
loss = loss_fn(preds , label)
loss.backward()
optimizer.step() #update weights

In [28]:
#Print Weight After model train
print(model.fc1.weight)

Parameter containing:
tensor([[-0.0324,  0.0190, -0.0261,  ..., -0.0228,  0.0319, -0.0240],
        [-0.0053, -0.0229,  0.0097,  ..., -0.0117, -0.0091,  0.0267],
        [-0.0293, -0.0009, -0.0311,  ...,  0.0178,  0.0032,  0.0210],
        ...,
        [ 0.0139, -0.0279,  0.0233,  ..., -0.0160, -0.0127, -0.0164],
        [-0.0024, -0.0433,  0.0123,  ..., -0.0349, -0.0179, -0.0192],
        [ 0.0087,  0.0249,  0.0175,  ..., -0.0080,  0.0095, -0.0137]],
       requires_grad=True)


In [29]:
#Print prediction
prd = model(feature)
print((label[0])) #lable for one data set

print(prd[0]) #NOTE: if Sigmoid is not used as final layer activation funciton ==> these prediction will not be range [0,1]. Which is wrong, as output lable in in range [0,1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([-1465.7537, -1509.0543, -1532.8311, -1506.2460, -1511.9248, -1465.7925,

# Train the model

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device} is used")
model.to(device)

cpu is used


MultiLabelClassifer(
  (fc1): Linear(in_features=876, out_features=4000, bias=True)
  (fc2): Linear(in_features=4000, out_features=4000, bias=True)
  (fc3): Linear(in_features=4000, out_features=4000, bias=True)
  (fc4): Linear(in_features=4000, out_features=4000, bias=True)
  (fc5): Linear(in_features=4000, out_features=206, bias=True)
  (drop): Dropout(p=0.4, inplace=False)
  (out): Linear(in_features=206, out_features=206, bias=True)
  (sig): Sigmoid()
  (Log_Sig): LogSigmoid()
  (soft): Softmax(dim=None)
)

In [31]:

import torch.optim as optim
learning_rate = [ .01]

for lr in learning_rate:
    optimizer = optim.Adam(model.parameters() , lr = lr)
    #criterion = torch.nn.BCELoss()
    criterion = nn.BCEWithLogitsLoss()
    #Difference b/w above two loss => https://discuss.pytorch.org/t/bceloss-vs-bcewithlogitsloss/33586
    
    
          
    for epoch in range(30):
        model.train()
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            features , labels  = batch
            
            features = features.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            
            preds = model(features)

            #print(sum(preds>1))
            loss = criterion((preds) , labels)


            loss.backward()
            optimizer.step() #update weights

            total_loss += loss.data.item()
            #print("total_loss: {}".format(total_loss))
            #total_correct  += get_num_correct(preds,labels)
        print(" epoch: {} , learning_rate:{} , total_loss: {}".format(epoch, lr, total_loss))

 epoch: 0 , learning_rate:0.01 , total_loss: 10.102628288790584
 epoch: 1 , learning_rate:0.01 , total_loss: 2.9404651364311576


In [48]:
def getPrediction(test):
    all_pred = np.zeros((test.shape[0] , 206))
    
    nfold = 1
    for i in range(nfold):
        input_size = train.shape[1]
        output_size = target.shape[1]
        
        hidden_dim = 1024
        
        #model = MultiLabelClassifer(input_size , hidden_dim , output_size)
        #model.load_state_dict() #use if want to load model from memory
        #model = MultiLabelClassifer(input_size,hidden_dim,output_size)
        #model.load_state_dict(torch.load(f"model{i}.bin"))
        
        predictions = list()
        
        test_tensor = torch.tensor(test , dtype = torch.float)
        test_dl  = torch.utils.data.DataLoader(test_tensor , batch_size = 64 )
        
        with torch.no_grad(): #stop weight update
            for i, inputs in enumerate(test_dl):
                inputs = inputs.to(device)
                outputs = model(inputs)
                predictions.extend(outputs.sigmoid().cpu().detach().numpy())
        
        all_pred += np.array(predictions) / nfold
    return all_pred



In [56]:
predictions = getPrediction(test)

## Calculate Loss

In [57]:
import torch.nn as nn
creteria = nn.BCEWithLogitsLoss()

## Final Accurray

In [58]:
import torch
print("predictions shap is {}, test_target {}".format(predictions.shape,test_target.shape))
test_target_np  =(test_target.iloc[:,1:]).astype(np.float32).to_numpy()
print("type: {}".format(type(test_target_np)))

#accurray_test = creteria( torch.from_numpy(predictions.astype(np.float32)) , torch.from_numpy(train_target.iloc[:,1:].astype(np.float32).to_numpy()))
accurray_test = creteria( torch.from_numpy(predictions.astype(np.float32)) , torch.from_numpy(test_target_np))
print("accurray on test data is: {}".format(accurray_test))

predictions shap is (4763, 206), test_target (4763, 207)
type: <class 'numpy.ndarray'>
accurray on test data is: 0.6944521069526672


In [43]:
#sample.iloc[:,1:] = predictions
#sample.to_csv('submission.csv',index=False)

In [ ]:
#sample.head()